#Libs and datasets

In [1]:
!pip install gdown

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for gdown: filename=gdown-4.5.1-py3-none-any.whl size=14933 sha256=18db2405e4993e186d78a0618b233b16c83e50f096adab36551d2dba38db4de8
  Stored in directory: c:\users\hitch\appdata\local\packages\pythonsoftwarefoundation.python.3.10_qbz5n2kfra8p0\localcache\local\pip\cache\wheels\38\72\83\dab5cb8321023eb1549d7a7d7f564b479ed4cf71edb70ddad6
Successfully built gdown


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
!gdown https://drive.google.com/uc?id=126dO4VNhLpYKT0TKp18RTAGjrHAl_ZpU -O mnist.npz
!gdown https://drive.google.com/uc?id=16wlkaf6GCGX0aJTOtzDo0ypnhqYZ7GVM -O kmnist.npz
!gdown https://drive.google.com/uc?id=1XMH39lcD2bnwy4AW3S-4-0Ge7JIYw6CF -O eurosat.npz
!gdown https://drive.google.com/uc?id=1BsfU84WJMRRKG3wzRZG6KuCperlLrxHc -O cifar10.npz
!gdown https://drive.google.com/uc?id=1MvPjY4m58TW51NZbUIl5tRJbERKMnIk7 -O pathmnist.npz
!gdown https://drive.google.com/uc?id=146WDl2VzVdLhnJl5JqYqKVyDPlDQoLDl -O octmnist.npz
!gdown https://drive.google.com/uc?id=1BIJFOn5ivB766qNIZdI2Owt8GAmpsmic -O organmnist_axial.npz

'gdown' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'gdown' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'gdown' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'gdown' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'gdown' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'gdown' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.
'gdown' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [3]:
from tensorflow.keras import callbacks, optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.applications import InceptionV3, ResNet50V2, EfficientNetB1, DenseNet169
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.layers import Dense, Flatten, Dropout, Lambda, Input
from tensorflow.image import resize
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from scipy.stats import friedmanchisquare, rankdata
import numpy as np
import pandas as pd
import tensorflow as tf
import re, os, time, requests
import matplotlib.pyplot as plt

C:\Users\hitch\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


#Configs

In [4]:
DATASETS = {

    # 'octmnist': { 'shape': (28, 28, 1), 'classes': 4,
    #     'phenotypes': [],
    # },
    'pathmnist': { 'shape': (28, 28, 3), 'classes': 9,
        'phenotypes': [],
    },
    # 'organmnist_axial': { 'shape': (28, 28, 1), 'classes': 11,
    #     'phenotypes': [''],
    # },
    # 'mnist': { 'shape': (28, 28, 1), 'classes': 10,
    #     'phenotypes': [],
    # },
    # 'cifar10': { 'shape': (32, 32, 3), 'classes': 10,
    #     'phenotypes': [],
    # },   
    # 'eurosat': { 'shape': (64, 64, 3), 'classes': 10,
    #     'phenotypes': [],
    # }, 

}
NUM_TRAINING = 3
BATCH_SIZE = 128
EPOCHS = 70

#Factories

In [5]:
def load_dataset(dataset_name):

  shape = DATASETS[dataset_name]['shape']
  dataset = np.load('%s.npz' % dataset_name, allow_pickle=True)

  if dataset_name == 'eurosat':
      
    print('eurosat')
    
    train = dataset['train'].tolist()

    train_images, train_labels = train['image'], train['label']

    train_images = train_images.reshape((train_images.shape[0], *shape))
    train_images = train_images.astype("float") / 255.0

    train_images, test_images, train_labels, test_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)
    validation_images, test_images, validation_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.2, random_state=42)

  elif dataset_name in ['pathmnist', 'octmnist', 'organmnist_axial']:
      
    print('medmnist:', dataset_name)
    
    train_images = dataset['train_images']
    validation_images = dataset['val_images']
    test_images = dataset['test_images']
    train_labels = dataset['train_labels']
    validation_labels = dataset['val_labels']
    test_labels = dataset['test_labels']

    if shape[2] == 1:
      train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
      validation_images = validation_images.reshape((validation_images.shape[0], 28, 28, 1))
      test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

    train_images = train_images.astype("float") / 255.0
    test_images = test_images.astype("float") / 255.0
    validation_images = validation_images.astype("float") / 255.0

  else:
      
    print('outros:', dataset_name)
    
    train = dataset['train'].tolist()
    test = dataset['test'].tolist()

    train_images, test_images, train_labels, test_labels = train['image'], test['image'], train['label'], test['label']

    train_images = train_images.reshape((train_images.shape[0], *shape))
    train_images = train_images.astype("float") / 255.0

    test_images = test_images.reshape((test_images.shape[0], *shape))
    test_images = test_images.astype("float") / 255.0

    validation_images, test_images, validation_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.2, random_state=42)

  lb = LabelBinarizer()
  train_labels = lb.fit_transform(train_labels)
  validation_labels = lb.transform(validation_labels)
  test_labels = lb.transform(test_labels)

  dataset.close()

  return train_images, train_labels, validation_images, validation_labels, test_images, test_labels

In [6]:
def f1_score(y_true, y_pred):
  
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  recall = true_positives / (possible_positives + K.epsilon())
  f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
  return f1_val

In [7]:
def build_model(dataset, phenotype):

    dataset_shape = DATASETS[dataset]['shape']
    dataset_classes = DATASETS[dataset]['classes']

    nconv, npool, nfc, nfcneuron = [int(i) for i in re.findall('\d+', phenotype.split('lr-')[0])]
    has_dropout = 'dropout' in phenotype
    has_batch_normalization = 'bnorm' in phenotype
    has_pool = 'pool' in phenotype
    learning_rate = float(phenotype.split('lr-')[1])

    # number of filters
    filter_size = 32

    model = models.Sequential()
    model.add(layers.InputLayer(input_shape=dataset_shape))

    # Pooling
    for i in range(npool):

        # Convolutions
        for j in range(nconv):

            model.add(layers.Conv2D(filter_size, (3, 3), activation='relu', padding='same'))

            # Duplicate number of filters for each two convolutions
            if (((i + j) % 2) == 1): filter_size = filter_size * 2

            # Add batch normalization
            if has_batch_normalization:
                model.add(layers.BatchNormalization())

        # Add pooling
        if has_pool:
            model.add(layers.MaxPooling2D(pool_size=(2, 2)))
            # Add dropout
            if has_dropout:
                model.add(layers.Dropout(0.25))

    model.add(layers.Flatten())

    # fully connected
    for i in range(nfc):
        model.add(layers.Dense(nfcneuron))
        model.add(layers.Activation('relu'))

    if has_dropout:
        model.add(layers.Dropout(0.5))

    model.add(layers.Dense(dataset_classes, activation='softmax'))

    opt = optimizers.Adam(learning_rate=learning_rate)

    return model, opt

#Train

In [8]:
def train_model(model, dataset):

  train_images, train_labels, validation_images, \
    validation_labels, test_images, test_labels = load_dataset(dataset)

  train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(BATCH_SIZE, drop_remainder=True)
  validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels)).batch(BATCH_SIZE, drop_remainder=True)

  accuracies, f1_scores = [], []

  for i in range(NUM_TRAINING):

    print('Training %s of %s' % (i + 1, NUM_TRAINING))

    history = model.fit(train_ds,
            epochs=EPOCHS, 
            validation_data=validation_ds,
            verbose=1)

    loss, accuracy, f1_score = model.evaluate(test_images, test_labels, verbose=1)

    print(accuracy, f1_score)

    accuracies.append(accuracy)
    f1_scores.append(f1_score)

  return {
      'accuracy': np.mean(accuracies),
      'accuracy_sd': np.std(accuracies),
      'f1_score': np.mean(f1_scores),
      'f1_score_sd': np.std(f1_scores),
  }

In [9]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

ValueError: Please provide a TPU Name to connect to.

In [ ]:
for dataset in DATASETS:

  for phenotype in DATASETS[dataset]['phenotypes']:

    print('DATASET:', dataset)
    print('PHENOTYPE:', phenotype)

    print('Building model.')

    model, opt = build_model(dataset, phenotype)

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', f1_score])
    # model.summary()

    print('Model created.')

    print('Begining training...')

    fitness = train_model(model, dataset)

    print('FITNESS:', fitness)

DATASET: cifar10
PHENOTYPE: (((conv*3)bnorm-pool-dropout)*3)fc*1*256*lr-0.001
Building model.
Model created.
Begining training...
outros: cifar10
Training 1 of 3
Epoch 1/70
390/390 [==============================] - 95s 236ms/step - loss: 1.8175 - accuracy: 0.3691 - f1_score: 0.2407 - val_loss: 2.4535 - val_accuracy: 0.2563 - val_f1_score: 0.1809
Epoch 2/70
390/390 [==============================] - 94s 238ms/step - loss: 1.2548 - accuracy: 0.5608 - f1_score: 0.5071 - val_loss: 1.0007 - val_accuracy: 0.6502 - val_f1_score: 0.6309
Epoch 3/70
390/390 [==============================] - 94s 238ms/step - loss: 1.0252 - accuracy: 0.6505 - f1_score: 0.6225 - val_loss: 0.8926 - val_accuracy: 0.6859 - val_f1_score: 0.6752
Epoch 4/70
390/390 [==============================] - 94s 238ms/step - loss: 0.8782 - accuracy: 0.7011 - f1_score: 0.6892 - val_loss: 0.8703 - val_accuracy: 0.7045 - val_f1_score: 0.6947
Epoch 5/70
390/390 [==============================] - 94s 239ms/step - loss: 0.8753 - accu